In [2]:
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"
:dep csv = "1.1"

In [5]:
use serde_json::Value;
use std::error::Error;
use std::fs::File;
use csv::Writer;

fn main() -> Result<(), Box<dyn Error>> {
    // Read the JSON file
    let file = File::open("input.json")?;  // Open the file
    let json_data: Value = serde_json::from_reader(file)?;  // Deserialize JSON


    // Create a CSV writer
    let mut wtr = Writer::from_writer(File::create("output.csv")?);

    // Check if the JSON is an array of objects (rows)
    if let Value::Array(records) = json_data {
        if records.is_empty() {
            return Err("JSON data is empty.".into());
        }

        // Extract headers from the first record (keys of the first JSON object)
        if let Value::Object(first_record) = &records[0] {
            // Write headers (keys)
            let headers: Vec<String> = first_record.keys().cloned().collect();
            wtr.write_record(&headers)?;


            // Write data for each record
            for record in records {
                // println!("Record: {:?}",record);
                if let Value::Object(record_obj) = record {
                    // Map through headers and extract the corresponding values
                    let row: Vec<String> = headers.iter()
                        .map(|key| {
                            // Check if the key exists, and if it does, get its value as a string
                            record_obj.get(key)
                                .and_then(|v| v.as_str())
                                .unwrap_or("")  // If the value is missing, use an empty string
                                .to_string()
                        })
                        .collect();
                    // Write the row to the CSV
                    wtr.write_record(row)?;
                }
            }
        }
    }

    // Flush and finalize the CSV writing
    wtr.flush()?;
    println!("CSV writing complete.");
    Ok(())
}


In [6]:
main()

CSV writing complete.


Ok(())